# 1. Analysis of the testing rate in comparison with the implementation of safety policies in South Korea

In [43]:
# Dependencies

import pandas as pd
import pandasql as ps
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, func


## Cleaning the data

In [17]:
# CSV files that are being analyzed

policy_csv = "Resources/Data/Policy.csv"
policy_df = pd.read_csv(policy_csv)
policy_df.head()

,policy_id,country,type,gov_policy,detail,start_date,end_date
0,1,Korea,Alert,Infectious Disease Alert Level,Level 1 (Blue),2020-01-03,2020-01-19
1,2,Korea,Alert,Infectious Disease Alert Level,Level 2 (Yellow),2020-01-20,2020-01-27
2,3,Korea,Alert,Infectious Disease Alert Level,Level 3 (Orange),2020-01-28,2020-02-22
3,4,Korea,Alert,Infectious Disease Alert Level,Level 4 (Red),2020-02-23,NaN
4,5,Korea,Immigration,Special Immigration Procedure,from China,2020-02-04,NaN


In [18]:
testing_csv = "Resources/Data/Time.csv"
testing_df = pd.read_csv(testing_csv)
testing_df.head()

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0
3,2020-01-23,16,22,21,1,0,0
4,2020-01-24,16,27,25,2,0,0


In [19]:
# New DataFrame for Policy.csv

new_policy = policy_df[["policy_id", "type", "detail", "start_date"]].copy()
policy_c = new_policy.rename(columns={"start_date": "date"})
policy_c

,policy_id,type,detail,date
0,1,Alert,Level 1 (Blue),2020-01-03
1,2,Alert,Level 2 (Yellow),2020-01-20
2,3,Alert,Level 3 (Orange),2020-01-28
3,4,Alert,Level 4 (Red),2020-02-23
4,5,Immigration,from China,2020-02-04
...,...,...,...,...
56,57,Transformation,On-site inspection of major logistics faciliti...,2020-05-29
57,58,Transformation,"Mandatory wearing of passenger mask domestic, ...",2020-05-27
58,59,Transformation,Drivers such as buses and taxis can refuse to ...,2020-05-26
59,60,Technology,new quick response (QR) code system this week ...,2020-06-10


In [20]:
# New dataFrame for time.csv (testing)

new_testing = testing_df[["date", "test", "negative", "confirmed", "released", "deceased"]].copy()
new_testing

,date,test,negative,confirmed,released,deceased
0,2020-01-20,1,0,1,0,0
1,2020-01-21,1,0,1,0,0
2,2020-01-22,4,3,1,0,0
3,2020-01-23,22,21,1,0,0
4,2020-01-24,27,25,2,0,0
...,...,...,...,...,...,...
158,2020-06-26,1232315,1200885,12602,11172,282
159,2020-06-27,1243780,1211261,12653,11317,282
160,2020-06-28,1251695,1219975,12715,11364,282
161,2020-06-29,1259954,1228698,12757,11429,282


### The table below, displays an average count of both confirmed and negative test results, as well as the number of patients that have been released, and the ones that have deceased. This allows us to see the effects of the implementation of policies to the virus testing rate. 

In [21]:
# (Set both dataframes into one, and clean it up)

avgresults_df = pd.merge(left=new_testing, right=policy_c, left_on='date', right_on='date')
del avgresults_df["policy_id"]
avgresults_df.head()

,date,test,negative,confirmed,released,deceased,type,detail
0,2020-01-20,1,0,1,0,0,Alert,Level 2 (Yellow)
1,2020-01-20,1,0,1,0,0,Technology,Patients Information
2,2020-01-28,116,97,4,0,0,Alert,Level 3 (Orange)
3,2020-02-04,607,462,16,0,0,Immigration,from China
4,2020-02-04,607,462,16,0,0,Health,1st EUA


In [22]:
clean_avgresults = avgresults_df.drop_duplicates(subset=['date'])
clean_results = clean_avgresults.dropna()
clean_results.head()

,date,test,negative,confirmed,released,deceased,type,detail
0,2020-01-20,1,0,1,0,0,Alert,Level 2 (Yellow)
2,2020-01-28,116,97,4,0,0,Alert,Level 3 (Orange)
3,2020-02-04,607,462,16,0,0,Immigration,from China
5,2020-02-12,5624,4811,28,7,0,Immigration,from Hong Kong
9,2020-02-23,26179,17520,602,18,6,Alert,Level 4 (Red)


## Setup connection between SQL and pandas

In [23]:
rds_connection_string = "postgres:postgres@localhost:5432/etl_project"

engine = create_engine(f'postgresql://{rds_connection_string}')

In [24]:
engine.table_names()

['testingpolicies']

In [26]:
# upload data to sql database
clean_results.to_sql(name='testingpolicies', con=engine, if_exists='append', index=False)

In [27]:
# Read updated database from SQL
pd.read_sql_query('SELECT * FROM testingpolicies', con = engine).head()

,date,test,negative,confirmed,released,deceased,type,detail
0,2020-01-20,1,0,1,0,0,Alert,Level 2 (Yellow)
1,2020-01-28,116,97,4,0,0,Alert,Level 3 (Orange)
2,2020-02-04,607,462,16,0,0,Immigration,from China
3,2020-02-12,5624,4811,28,7,0,Immigration,from Hong Kong
4,2020-02-23,26179,17520,602,18,6,Alert,Level 4 (Red)


In [28]:
# policies query: look at the amount of policies implemented

p_qr = """SELECT type, detail FROM clean_results"""
print(ps.sqldf(p_qr,locals()))

              type                                             detail
0            Alert                                   Level 2 (Yellow)
1            Alert                                   Level 3 (Orange)
2      Immigration                                         from China
3      Immigration                                     from Hong Kong
4            Alert                                      Level 4 (Red)
5           Health                                by Local Government
6           Health                                            3rd EUA
7           Social                                             Strong
8        Education                        Daycare Center for Children
9           Health                      Standard Operating Procedures
10      Technology                      Public Mask Sales Information
11     Immigration                                         from Japan
12     Immigration                                         from Italy
13          Health  

In [29]:
# Rate and count of positive and negative results based on the date of policies implementation 
test_results =  ps.sqldf("SELECT test, negative, confirmed, date, detail FROM clean_results")
test_results.head(20)

,test,negative,confirmed,date,detail
0,1,0,1,2020-01-20,Level 2 (Yellow)
1,116,97,4,2020-01-28,Level 3 (Orange)
2,607,462,16,2020-02-04,from China
3,5624,4811,28,2020-02-12,from Hong Kong
4,26179,17520,602,2020-02-23,Level 4 (Red)
5,53553,31576,1261,2020-02-26,by Local Government
6,66652,39318,1766,2020-02-27,3rd EUA
7,94055,55723,3150,2020-02-29,Strong
8,109591,71580,4212,2020-03-02,Daycare Center for Children
9,136707,102965,5328,2020-03-04,Standard Operating Procedures


In [30]:
# Number of released and deceased patients, based on the policies implementation date
release_qr = ps.sqldf('SELECT test, released, deceased, date FROM clean_results')
release_qr.head(20)

,test,released,deceased,date
0,1,0,0,2020-01-20
1,116,0,0,2020-01-28
2,607,0,0,2020-02-04
3,5624,7,0,2020-02-12
4,26179,18,6,2020-02-23
5,53553,24,12,2020-02-26
6,66652,26,13,2020-02-27
7,94055,28,17,2020-02-29
8,109591,31,22,2020-03-02
9,136707,41,32,2020-03-04


# 2. Comparison ratio of infection per region based on tracing of infection: group or individual. 

In [32]:
# Read csv files
case_path = "Resources/Data/Case.csv"
Case = pd.read_csv(case_path)
# Print head of data
Case.head()


,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Yongsan-gu,True,Itaewon Clubs,139,37.538621,126.992652
1,1000002,Seoul,Gwanak-gu,True,Richway,119,37.48208,126.901384
2,1000003,Seoul,Guro-gu,True,Guro-gu Call Center,95,37.508163,126.884387
3,1000004,Seoul,Yangcheon-gu,True,Yangcheon Table Tennis Club,43,37.546061,126.874209
4,1000005,Seoul,Dobong-gu,True,Day Care Center,43,37.679422,127.044374


In [35]:

# Read csv files
region_path = "Resources/Data/Region.csv"
Region = pd.read_csv(region_path)
# Print data
Region

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080
...,...,...,...,...,...,...,...,...,...,...,...,...
239,61160,Gyeongsangnam-do,Haman-gun,35.272481,128.406540,16,20,0,1.19,23.74,14.7,94
240,61170,Gyeongsangnam-do,Hamyang-gun,35.520541,127.725177,13,12,0,1.01,32.65,20.9,83
241,61180,Gyeongsangnam-do,Hapcheon-gun,35.566702,128.165870,17,15,0,0.71,38.44,24.7,96
242,70000,Jeju-do,Jeju-do,33.488936,126.500423,113,123,4,1.53,15.10,6.4,1245


In [37]:
Case.columns

Index(['code', 'province', 'city', 'latitude', 'longitude',
       'elementary_school_count', 'kindergarten_count', 'university_count',
       'academy_ratio', 'elderly_population_ratio', 'elderly_alone_ratio',
       'nursing_home_count'],
      dtype='object')

In [38]:
Region.columns

Index(['code', 'province', 'city', 'latitude', 'longitude',
       'elementary_school_count', 'kindergarten_count', 'university_count',
       'academy_ratio', 'elderly_population_ratio', 'elderly_alone_ratio',
       'nursing_home_count'],
      dtype='object')

In [39]:
# Merge dataframes based on City name
CompleteDataDFCity = pd.merge(Case, Region, how ="left", on = ["city", "city"])

CompleteDataDFCity

,case_id,province_x,city,group,infection_case,confirmed,latitude_x,longitude_x,code,province_y,latitude_y,longitude_y,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,1000001,Seoul,Yongsan-gu,True,Itaewon Clubs,139,37.538621,126.992652,10210.0,Seoul,37.532768,126.990021,15.0,13.0,1.0,0.68,16.87,6.5,435.0
1,1000002,Seoul,Gwanak-gu,True,Richway,119,37.48208,126.901384,10050.0,Seoul,37.478290,126.951502,22.0,33.0,1.0,0.89,15.12,4.9,909.0
2,1000003,Seoul,Guro-gu,True,Guro-gu Call Center,95,37.508163,126.884387,10070.0,Seoul,37.495632,126.887650,26.0,34.0,3.0,1.00,16.21,5.7,741.0
3,1000004,Seoul,Yangcheon-gu,True,Yangcheon Table Tennis Club,43,37.546061,126.874209,10190.0,Seoul,37.517189,126.866618,30.0,43.0,0.0,2.26,13.55,5.5,816.0
4,1000005,Seoul,Dobong-gu,True,Day Care Center,43,37.679422,127.044374,10100.0,Seoul,37.668952,127.047082,23.0,26.0,1.0,0.95,17.89,7.2,485.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,6100012,Gyeongsangnam-do,-,False,etc,20,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,7000001,Jeju-do,-,False,overseas inflow,14,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,7000002,Jeju-do,-,False,contact with patient,0,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,7000003,Jeju-do,-,False,etc,4,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Clean merged Data
CleanDF = CompleteDataDFCity[["city", "group", "infection_case", "confirmed", "code"]].copy()
CleanDF

,city,group,infection_case,confirmed,code
0,Yongsan-gu,True,Itaewon Clubs,139,10210.0
1,Gwanak-gu,True,Richway,119,10050.0
2,Guro-gu,True,Guro-gu Call Center,95,10070.0
3,Yangcheon-gu,True,Yangcheon Table Tennis Club,43,10190.0
4,Dobong-gu,True,Day Care Center,43,10100.0
...,...,...,...,...,...
218,-,False,etc,20,NaN
219,-,False,overseas inflow,14,NaN
220,-,False,contact with patient,0,NaN
221,-,False,etc,4,NaN


In [44]:
# drop NAn Values
CleanDF.replace(np.nan, 0, inplace=True)
CleanDF

,city,group,infection_case,confirmed,code
0,Yongsan-gu,True,Itaewon Clubs,139,10210.0
1,Gwanak-gu,True,Richway,119,10050.0
2,Guro-gu,True,Guro-gu Call Center,95,10070.0
3,Yangcheon-gu,True,Yangcheon Table Tennis Club,43,10190.0
4,Dobong-gu,True,Day Care Center,43,10100.0
...,...,...,...,...,...
218,-,False,etc,20,0.0
219,-,False,overseas inflow,14,0.0
220,-,False,contact with patient,0,0.0
221,-,False,etc,4,0.0


In [45]:
# Reassuring table is clean
CleanDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223 entries, 0 to 222
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            223 non-null    object 
 1   group           223 non-null    bool   
 2   infection_case  223 non-null    object 
 3   confirmed       223 non-null    int64  
 4   code            223 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 8.9+ KB


In [47]:
# Drop duplicates in code column
CleanDF_bis = CleanDF.drop_duplicates(subset = ["code"])
CleanDF_biss = CleanDF_bis.drop_duplicates(subset = ["city"])
Cleancheck = CleanDF_biss.dropna()
Cleancheck.head()

,city,group,infection_case,confirmed,code
0,Yongsan-gu,True,Itaewon Clubs,139,10210.0
1,Gwanak-gu,True,Richway,119,10050.0
2,Guro-gu,True,Guro-gu Call Center,95,10070.0
3,Yangcheon-gu,True,Yangcheon Table Tennis Club,43,10190.0
4,Dobong-gu,True,Day Care Center,43,10100.0


In [48]:
# Uploading dataframe into SQL database
Cleancheck.to_sql(name='regions', con=engine, if_exists='append', index=False)


In [49]:
# Read updated db from SQL
pd.read_sql_query('SELECT * from regions', con=engine).head(20)

,city,group,infection_case,confirmed,code
0,Yongsan-gu,True,Itaewon Clubs,139,10210.0
1,Gwanak-gu,True,Richway,119,10050.0
2,Guro-gu,True,Guro-gu Call Center,95,10070.0
3,Yangcheon-gu,True,Yangcheon Table Tennis Club,43,10190.0
4,Dobong-gu,True,Day Care Center,43,10100.0
5,from other city,True,SMR Newly Planted Churches Group,36,0.0
6,Dongdaemun-gu,True,Dongan Church,17,10110.0
7,Eunpyeong-gu,True,Eunpyeong St. Mary's Hospital,14,10220.0
8,Seongdong-gu,True,Seongdong-gu APT,13,10160.0
9,Jongno-gu,True,Jongno Community Center,10,10230.0


In [53]:

# Perform query that shows whether an infection is a cluster or just an individual case
clusterqr= ps.sqldf("""SELECT infection_case, city FROM Cleancheck""")
clusterqr.head()

,infection_case,city
0,Itaewon Clubs,Yongsan-gu
1,Richway,Gwanak-gu
2,Guro-gu Call Center,Guro-gu
3,Yangcheon Table Tennis Club,Yangcheon-gu
4,Day Care Center,Dobong-gu


# 3. Analysis of COVID-19 confirmed cases in South Korea to determine which age group and gender was most affected. And trying to find if there is a clear distinction in ‘number of infections’ or ‘death rate’ between different age groups or genders, if there is one.

In [54]:

# Store CSV into DataFrame
csv_file = "Resources/Data/PatientInfo.csv"
patient_info_df = pd.read_csv(csv_file)
patient_info_df.head()

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,male,50s,Korea,Seoul,Gangseo-gu,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,male,30s,Korea,Seoul,Jungnang-gu,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,male,50s,Korea,Seoul,Jongno-gu,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,male,20s,Korea,Seoul,Mapo-gu,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,female,20s,Korea,Seoul,Seongbuk-gu,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released


In [55]:
patient_info_df.columns

Index(['patient_id', 'sex', 'age', 'country', 'province', 'city',
       'infection_case', 'infected_by', 'contact_number', 'symptom_onset_date',
       'confirmed_date', 'released_date', 'deceased_date', 'state'],
      dtype='object')

In [56]:
# new DF with select columns

new_patient_info_df = patient_info_df[['patient_id', 'sex', 'age', 'province', 'city', \
                                       'confirmed_date', 'released_date', 'state']].copy()
new_patient_info_df

,patient_id,sex,age,province,city,confirmed_date,released_date,state
0,1000000001,male,50s,Seoul,Gangseo-gu,2020-01-23,2020-02-05,released
1,1000000002,male,30s,Seoul,Jungnang-gu,2020-01-30,2020-03-02,released
2,1000000003,male,50s,Seoul,Jongno-gu,2020-01-30,2020-02-19,released
3,1000000004,male,20s,Seoul,Mapo-gu,2020-01-30,2020-02-15,released
4,1000000005,female,20s,Seoul,Seongbuk-gu,2020-01-31,2020-02-24,released
...,...,...,...,...,...,...,...,...
5160,7000000015,female,30s,Jeju-do,Jeju-do,2020-05-30,2020-06-13,released
5161,7000000016,NaN,NaN,Jeju-do,Jeju-do,2020-06-16,2020-06-24,released
5162,7000000017,NaN,NaN,Jeju-do,Jeju-do,2020-06-18,NaN,isolated
5163,7000000018,NaN,NaN,Jeju-do,Jeju-do,2020-06-18,NaN,isolated


# Clean DataFrame

In [57]:

# Replacing 'Nan' values with '0'
new_patient_info_df.fillna(0, inplace=True)


In [58]:
# Checking for null values in DataFrame
new_patient_info_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5165 entries, 0 to 5164
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   patient_id      5165 non-null   int64 
 1   sex             5165 non-null   object
 2   age             5165 non-null   object
 3   province        5165 non-null   object
 4   city            5165 non-null   object
 5   confirmed_date  5165 non-null   object
 6   released_date   5165 non-null   object
 7   state           5165 non-null   object
dtypes: int64(1), object(7)
memory usage: 322.9+ KB


In [59]:
# Checking for duplicates in a column
new_patient_info_df[new_patient_info_df["patient_id"].duplicated()]


,patient_id,sex,age,province,city,confirmed_date,released_date,state
1555,1200012238,female,20s,Daegu,Nam-gu,2020-06-17,0,isolated


In [60]:
# Locate the values that are duplicated and compare both the original and the copy
find = new_patient_info_df[new_patient_info_df['patient_id'].astype(str).str.contains('1200012238')]
find

,patient_id,sex,age,province,city,confirmed_date,released_date,state
1547,1200012238,female,20s,Daegu,Icheon-dong,2020-06-17,0,isolated
1555,1200012238,female,20s,Daegu,Nam-gu,2020-06-17,0,isolated


In [61]:
cleaned_info = new_patient_info_df.drop_duplicates(subset = ["patient_id"], keep=False)
cleaned_info

,patient_id,sex,age,province,city,confirmed_date,released_date,state
0,1000000001,male,50s,Seoul,Gangseo-gu,2020-01-23,2020-02-05,released
1,1000000002,male,30s,Seoul,Jungnang-gu,2020-01-30,2020-03-02,released
2,1000000003,male,50s,Seoul,Jongno-gu,2020-01-30,2020-02-19,released
3,1000000004,male,20s,Seoul,Mapo-gu,2020-01-30,2020-02-15,released
4,1000000005,female,20s,Seoul,Seongbuk-gu,2020-01-31,2020-02-24,released
...,...,...,...,...,...,...,...,...
5160,7000000015,female,30s,Jeju-do,Jeju-do,2020-05-30,2020-06-13,released
5161,7000000016,0,0,Jeju-do,Jeju-do,2020-06-16,2020-06-24,released
5162,7000000017,0,0,Jeju-do,Jeju-do,2020-06-18,0,isolated
5163,7000000018,0,0,Jeju-do,Jeju-do,2020-06-18,0,isolated


In [62]:
# Upload the dataframe into the SQL database
cleaned_info.to_sql(name='patient_info', con=engine, if_exists='append', index=False)